In [533]:
import operator

%runfile Parser.sage
%runfile Parsers.sage

In [356]:
pWait = lambda f: empty() >> (lambda _: f())

In [424]:
def pContains(token, string):
    return len(parse(many(pNext()) + token + many(pNext()), string, returnList = True)) > 0

In [538]:
exhaust = lambda token, s: liftPP(lambda pre, res, post: pReturn(res) if pre == s + post else pError(), pLook(), token, pLook())

In [542]:
parse(exhaust(text('hi'), 'hi'), "hi")

'hi'

In [567]:
def sepParse(parser, string, **kwargs):
    return reduce(lambda accum, other: accum | pReturn(other), parse(parser, string, returnList=True, **kwargs), pError())

In [577]:
print sepParse(pReturn(1) | pReturn(2), "")

((error) or (return (1))) or (return (2))


In [568]:
ppAccept = lambda token, checkprec: lambda prec: pWait(token) if prec == checkprec else pError()
ppAcceptF = lambda token, checkprec: lambda prec: empty() >> (lambda _: token(prec)) if prec == checkprec else pError()

In [569]:
infix = lambda token, symbol, checkprec, op: ppAcceptF(lambda prec: pLook() >> (lambda look: pError() if not pContains(symbol, look) else liftP(lambda l,_, r: op(l,r), token(prec), pSpaces + symbol + pSpaces, token(prec + 1))), checkprec)

In [584]:
def infix(token, symbol, checkprec, op):
    return ppAcceptF(lambda prec: (many(pNext()) < (pSpaces + symbol + pSpaces)) >> (lambda fst: (sepParse(pWait(token(prec)), ''.join(fst)) & pWait(token(prec))) >> (lambda t: pReturn(op(*t)))), checkprec) 

In [601]:
maxprec = 10
token = lambda prec: lambda: pError() if prec > maxprec else pWait(token(prec + 1)) \
            | ppAccept(lambda: pNatural, maxprec)(prec) \
            | ppAccept(lambda: paren(pWait(token(0))), maxprec)(prec) \
            | infix(token, text('+'), 5, lambda l, r: ('+', l, r))(prec) \
            | infix(token, text('-'), 6, lambda l, r: ('-', l, r))(prec) \

token = lambda prec: lambda: pError() if prec > maxprec else pWait(token(prec + 1)) \
            | ppAccept(lambda: pNatural, maxprec)(prec) \
            | ppAccept(lambda: paren(pWait(token(0))), maxprec)(prec) \
            | infix(token, text('+'), 5, operator.add)(prec) \
            | infix(token, text('-'), 6, operator.sub)(prec) \

In [602]:
tokenr = token(0)()

In [605]:
parse(tokenr, "((10) + 10) - (10 + 10)")

0

In [565]:
parse(many(pNext()) < (pSpaces + text('+') + pSpaces), "10 + 10", mustFinish=False)

ValueError: Ambiguous parse! Using parser 'return many next if then ((many whitespace) then (text +)) then (many whitespace)' on '10 + 10' returned [['1', '0', ' '], ['1', '0', ' '], ['1', '0'], ['1', '0']]

In [583]:
parse((many(pNext()) < (pSpaces + text('+') + pSpaces)) >> (lambda fst: (sepParse(pNatural, ''.join(fst)) & pNatural)), "10 + 10")

(10, 10)

In [582]:
parses((many(pNext()) < (pSpaces + text('+') + pSpaces)), "10 + 10")

[(['1', '0', ' '], '10'),
 (['1', '0', ' '], ' 10'),
 (['1', '0'], '10'),
 (['1', '0'], ' 10')]

In [575]:
parse(infix(lambda prec: pNatural, text('+'), 10, operator.add)(10), "10 + 16")

TypeError: 'Parser' object is not callable

In [547]:
parse(paren(infix(lambda prec: pNatural, text('+'), 10, operator.add)(10)), "(10 + 10)")

TypeError: 'Parser' object is not callable

In [16]:
pSum = lambda term: term | liftP(lambda t, _1, op, _2, s: {'+': t + s, '-': t - s}[op], term, pSpaces, text('+') | text('-'), pSpaces, lambda: pSum(term))

In [14]:
parse(pSum(pNatural), "1 + 2 - 3 + 5")

-5

In [ ]:
parse(precParser([(pNatural, 10)], [(paren, 0)], [(text('+'), 5, 'right')]), )

In [4]:
listP = lambda n: between('[', ']', 
                manyWithEnd(
                      n >> (lambda n:
                      text(', ') + pReturn(n)),
                      n
                ))

In [5]:
digitsRange = lambda start, end: reduce(operator.or_, [expect(str(d)) + pReturn(d) for d in range(start, end)])

In [6]:
digits = digitsRange(0, 10)

In [7]:
natural = (some(digits) >> (lambda digs: pReturn(sum([ n * 10^e for e, n in enumerate(digs[::-1])])))).setname('natural')

In [8]:
integer = (natural | (expect('-') + natural >> (lambda n: pReturn(-n)))).setname('integer')

In [9]:
posrationalDecimals = liftP(lambda integral, _, decimals: integral + sum([n * (10 ** (-e - 1)) for e, n in enumerate(decimals)]), natural, text('.'), some(digits))
posrationalDivision = liftP(lambda numerator, _, denominator: numerator / denominator, natural, text('/'), natural)

posrational = (natural | posrationalDecimals | posrationalDivision).setname('positive rational')

In [10]:
rational = (posrational | (expect('-') + posrational >> (lambda n: pReturn(-n)))).setname('rational')

In [11]:
complexTerm = (rational | liftP(lambda r, _: r * I, rational, expect('i') | expect('I') | text('*i') | text('*I')) | 
               ((text('i') | text('I')) + pReturn(I)) | ((text('-i') | text('-I')) + pReturn(-I))).setname('complex term')

sumordiff = lambda parser: (liftP(lambda l, _, r: l + r, parser, many(expect(' ')) + text('+') + many(expect(' ')), parser) | 
                            liftP(lambda l, _, r: l - r, parser, many(expect(' ')) + text('-') + many(expect(' ')), parser)).setname('sum or difference of ' + str(parser))

complexP = (complexTerm | sumordiff(complexTerm)).setname('complex')

In [12]:
tannakian_set = between('{', '}', delimited(complexP, ',')) | (text('Ø') + pReturn([]))


symbolify = lambda xs, m: [(x, m) for x in xs]

tannakian_symbol = (
    tannakian_set >> (lambda x:
    expect('/') +
    tannakian_set >> (lambda y:
    pReturn(symbolify(x, 1) + symbolify(y, -1))
    ))
)

In [13]:
parse(tannakian_symbol, '{1, -i + 1/2}/{ -1}')

[(1, 1), (-I + 1/2, 1), (-1, -1)]

In [14]:
tree = lambda n: (n | liftP(lambda _0, left, _1, op, _2, right, _3: (op, left, right), expect('('), lambda: tree(n), many(expect(' ')), text('+') | text('-') | text('*'), many(expect(' ')), lambda: tree(n), expect(')')))

In [110]:
parse(tree(natural), "((1 + (1 - 3)) * (2 + 2))")

('*', ('+', 1, ('-', 1, 3)), ('+', 2, 2))

In [16]:
def eval_tree(tree):
    try:
        return int(tree)
    except:
        op, left, right = tree
        return {
            '+': operator.add,
            '-': operator.sub,
            '*': operator.mul
        }[op](eval_tree(left), eval_tree(right))

In [17]:
eval_tree(parse(tree(natural), "((1 + 2) * (2 + 3))"))

15

In [18]:
parse(delimitedWith(lambda d: natural >> (lambda res: pReturn(res + d)), text(' ') + natural < text(' '), 0), "  51   4   12  88 9 9 2")

[51, 16, 97, 11]

In [19]:
parse((natural < text(' ')) & natural, "12 2")

(12, 2)

In [20]:
parse(delimited(natural, text(' '), paddingSpaces = False), "51 4 12 88 9 9 2")

[51, 4, 12, 88, 9, 9, 2]

In [21]:
parse(delimited(natural, expect(',')), "1, 2")

[1, 2]

In [22]:
parse(text('t') + pReturn(1) | text('t') + pReturn(2), "t")

ValueError: Ambiguous parse! Using parser '((text t) then (return (1))) or ((text t) then (return (2)))' on 't' returned [1, 2]

In [23]:
parse(delimited(natural, expect(','), some = True), "1, 3")

[1, 3]

In [24]:
polynomial = lambda n: delimitedWith(lambda sign: term(n) >> (lambda t: pReturn(sign * t)), valuePair('+', 1) | valuePair('-', -1), 1) >> (lambda res: pReturn(sum(res)))
term = lambda n: empty() >> (lambda _: n | parenss(polynomial(n)))

In [25]:
parse(polynomial(natural), "1 - 2 + (6 + 2) - (3 + 4)")


0

In [26]:
str(polynomial(natural))

"(many <function <lambda> at 0x7fd566c2b8c0> delimitedWith by (value pair '+' to '1') or (value pair '-' to '-1') (first value 1)) then value to (<function <lambda> at 0x7fd567e33f50>)"

In [27]:
parse(polynomial(complexTerm), "((1 - 2)) + ((59676 - 2) + 1)")

59674

In [28]:
parse(natural < many(expectLetter()) + text('end'), "12DAFGDXfdgsend")

12

In [117]:
# Bind to this to get access to the identifier scope
getScope = empty() >> (lambda _, scope=None: pReturn(scope))

identifier = some(expectLetter()) >> (lambda l: pReturn(''.join(l)))
new_identifier = getScope >> (lambda scope: identifier >> (lambda text: pFail() if text in scope else pReturn(text)))

expression = identifier | 

assignment = new_identifier & many(expectSpace()) + text(":=") + many(expectSpace()) + expression

lines = liftP(lambda content, rest: [content] + rest, empty() | assignment, lambda: pReturn([]) | text('\n') + lines)
code = lines

def compileLanguage(inp):
    scope = {}
    return parse(code, inp, scope = {'scope': scope})
    

SyntaxError: invalid syntax (<ipython-input-117-75c85a3e82c4>, line 7)

In [116]:
inp = """
Hey := 5
Wow := 4
JEG := 3
"""

print inp
compileLanguage(inp)


Hey := 5
Wow := 4
JEG := 3



[(), ('Hey', 5), ('Wow', 4), ('JEG', 3), ()]

In [40]:
''.join(parse(some(expectLetter()), "Laugh"))

'Laugh'

In [109]:
parse(tannakian_symbol, "{1, 2}/{1 - i}")

[(1, 1), (2, 1), (-I + 1, -1)]

In [17]:
consonant = expectBy(lambda c: c in string.ascii_letters and c not in "laeiouyAEIOUY")
vowel = expectBy(lambda c: c in "aeiouyAEIOUY")

In [21]:
parse(many(expectPunctuation() | text('l') | text('L') | (text('l') + vowel < text('l')) | (consonant >> (lambda c: text('o') + text(c)))) >> (lambda ar: pReturn(''.join(ar))), "nonlillolL.", mustFinish=True)

'nioL.'